# **Εργασία 1**

Παλινδρόμηση με νευρωνικά δίκτυα και το climate change dataset.

## Φόρτωση και προεργασία

Φόρτωση βιβλιοθηκών, φόρτωση και απεικόνιση δεδομένων

In [ ]:
# Εισάγουμε όλες τις απαραίτητες βιβλιοθήκες
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [ ]:
# Φορτώνουμε το αρχείο CSV με τη βιβλιοθήκη Pandas μόνο για λόγους απεικόνισης
dataframe = pd.read_csv('climate_change.csv')
print(dataframe.head(5))
print("Διαστάσεις:", dataframe.shape)

   Year  Month    MEI     CO2      CH4      N2O   CFC-11   CFC-12        TSI  \
0  1983      5  2.556  345.96  1638.59  303.677  191.324  350.113  1366.1024   
1  1983      6  2.167  345.52  1633.71  303.746  192.057  351.848  1366.1208   
2  1983      7  1.741  344.15  1633.22  303.795  192.818  353.725  1366.2850   
3  1983      8  1.130  342.25  1631.35  303.839  193.602  355.633  1366.4202   
4  1983      9  0.428  340.17  1648.40  303.901  194.392  357.465  1366.2335   

   Aerosols   Temp  
0    0.0863  0.109  
1    0.0794  0.118  
2    0.0731  0.137  
3    0.0673  0.176  
4    0.0619  0.149  
Διαστάσεις: (308, 11)


## Δημιουργία custom αντικειμένου Dataset

Δημιουργούμε ένα αντικείμενο Dataset το οποίο θα τροφοδοτήσει τους dataloaders, και ένα αντικείμενο transform που θα τροποποιεί τα δεδομένα πριν διαβαστούν από τους loaders.

Στο μέλλον το transform θα είναι πολύ σημαντικό για το data augmentation, αλλά για την ώρα το χρησιμοποιούμε για την κανονικοποίηση των δεδομένων.

In [ ]:
class ClimateDataset(torch.utils.data.Dataset):
  def __init__(self, csv_file, transform=None, train=True):
    """
    Args:
      csv_file (string): Path to the csv file
      transform (callable, optional): Optional transform to be applied
          on each sample.
      train (bool): whether to create the training set or the test set
    """

    # Ορίζουμε το μέγεθος του training set
    train_set_size = 250

    # Φόρτωση του csv μέσω Pandas, διαγραφή στηλών που δε βοηθούν στην ανάλυση,
    # μετατροπή των Pandas Dataframes σε Numpy Arrays, που έχουν πιο βοηθητικές
    # ιδιότητες
    dataframe = pd.read_csv(csv_file)
    targets = np.array(dataframe[['Temp']]).astype('float32')
    data = np.array(dataframe.drop(columns=['Year','Month','Temp'])).astype('float32')

    # Διαχωρισμός των δεδομένων σε training και test set
    # Αυτή η γραμμή θα πρέπει στο μέλλον να αντικατασταθεί αν θέλουμε να κάνουμε 
    # πιο ρεαλιστική αξιολόγηση
    # Θέτουμε και το seed στο random state για λόγους επαναληψιμότητας
    train_data, test_data, train_targets, test_targets = train_test_split(data, 
                            targets, train_size=train_set_size, random_state=665)

    # Θέτουμε και αρχικοποιούμε το transform
    self.transform = transform
    # Προσοχή, το transform αρχικοποιείται με τις τιμές του training set, είτε
    # το Dataset αφορά το train είτε το test set.
    self.transform.fit(train_data, train_targets)

    # Αν μας έχει ζητηθεί να δημιουργηθεί train database, κρατάμε τα train data.
    # Αλλιώς κρατάμε τα test data. 
    if train==True:
      self.data = train_data
      self.targets = train_targets
    else:
      self.data = test_data
      self.targets = test_targets
    # Η αρχικοποίηση τελειώνει εδώ

  # Ή __len__ ενός αντικειμένου Dataset οφείλει να επιστρέφει το πλήθος των
  # αντικειμένων του.
  def __len__(self):
    return len(self.targets)

  # Η __getitem__ ενός αντικειμένου Dataset παίρνει ως παράμετρο ένα index και 
  # επιστρέφει το αντίστοιχο αντικείμενο. Τη χρησιμοποιεί ο DataLoader για να 
  # αντλεί minibatches.
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()
    item_data = self.data[idx,:]
    item_target = self.targets[idx,:]
    
    # Αν έχουμε ορίσει transform, το εφαρμόζουμε στο αντικείμενο πριν το 
    # επιστρέψουμε. Όταν καλώ το transform με το όνομά του, στην πραγματικότητα
    # καλείται η transform.__call__() (βλ. παρακάτω)
    if self.transform:
      item_data, item_target = self.transform(item_data, item_target)
    return item_data, item_target


# Η κλάση αυτή θα παίξει το ρόλο του Transform. Θα αφαιρεί την ελάχιστη τιμή
# από κάθε στήλη των δεδομένων και θα διαιρεί με το φάσμα της, ώστε να κανονικοποιεί
# τις τιμές όλων των μεταβλητών (και των στόχων) στο [0,1]
class MinMaxScaler():

  # H fit αρχικοποιείται με τα training data, και υπολογίζει τις ελάχιστες και 
  # μέγιστες τιμές. Τα αντικείμενα είναι numpy arrays, οπότε έχουν ενσωματωμένη
  # τη συνάρτηση min και max 
  def fit(self, data, targets):
    self.data_min = data.min(0, keepdims=True)
    self.target_min = targets.min(0, keepdims=True)
    self.data_max = data.max(0, keepdims=True)
    self.target_max = targets.max(0, keepdims=True)
    return self

  # Όταν καλείται η transform.__call__(), δέχεται ένα αντικείμενο (data και target)
  # και το επιστρέφει μετασχηματισμένο -εν προκειμένω κανονικοποιημένο
  def __call__(self, data, target):
    data = (data - self.data_min)/(self.data_max-self.data_min)
    target = (target - self.target_min)/(self.target_max-self.target_min)
    return data, target

  # Θα εκπαιδεύσουμε ένα σύστημα να μαθαίνει τα κανονικοποιημένα targets. Για
  # εφαρμογή στον πραγματικό κόσμο, θα χρειαστεί να μπορούμε να μετασχηματίζουμε
  # τα outputs του συστήματος πίσω στο αρχικό φάσμα τιμών του
  def inverse_transform(self, data, target):
    data=data * (self.data_max-self.data_min) + self.data_min
    target = target * (self.target_max-self.target_min) + self.target_min
    return data, target

## Δημιουργία DataLoaders

Δημιουργούμε dataloaders για το training set και για το test set.

In [ ]:
# To batch size είναι αρκετά μεγάλο ώστε να χωράει όλα τα training data και όλα 
# τα test data σε ένα loop (πρακτικά εφαρμόζουμε Batch Gradient Descent, κάθε
# Batch είναι ένα Epoch) 
# (αν το batch size είναι μεγαλύτερο από τα διαθέσιμα δεδομένα, η pytorch 
# δημιουργεί ένα batch με τα διαθέσιμα δεδομένα)
batch_size = 500
transform=MinMaxScaler()

train_dataset = ClimateDataset(csv_file='climate_change.csv', train=True, transform=transform)
test_dataset = ClimateDataset(csv_file='climate_change.csv', train=False, transform=transform)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)


## Αρχιτεκτονική δικτύου

Ορίζουμε ένα απλό δίκτυο με έναν νευρώνα για γραμμική παλινδρόμηση

In [ ]:
# Κώδικας ορισμού του δικτύου
class NeuralNetwork1(nn.Module):
  def __init__(self):
      # Τρέχω την init της κλάσης-γονιού
      super(NeuralNetwork1, self).__init__()
      
      # Δημιουργώ μια μέθοδο Flatten που θα μετατρέπει το Tensor εισόδου σε
      # μονοδιάστατο
      self.flatten = nn.Flatten()

      # Η κλάση Sequential() είναι ένα container μέσα στο οποίο βάζουμε τα
      # Module() που θα χτίζουν τα layers του δικτύου 
      self.network_architecture = nn.Sequential(
          nn.Linear(8,1)
      ) 

  # Η μέθοδος forward απλώς μετατρέπει τα inputs σε διάνυσμα και τα περνάει
  # από τη network_architecture που δημιουργήσαμε
  def forward(self, x):
      x = self.flatten(x)
      logits = self.network_architecture(x)
      return logits


Μετα την προέργασια απο την κλαση το input χ του δικτύου αποτελείται απο 250 μεταβλητες και 8 κατηγοριες  

## Training και Test Loop

Γράφουμε τον κώδικα του training loop και του test loop

In [ ]:

def train_loop(dataloader, model, loss_fn, optimizer, display):
    #για το μεγεθος
    size = len(dataloader.dataset)

    for batch, (X, y) in enumerate(dataloader):

      pred = model(X)
      y = np.reshape(y,(250,1))
      loss = loss_fn(pred, y)

      # Backpropagation
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      if np.mod(display , 100 ) == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss  = 0

    with torch.no_grad():
      for X, y in dataloader:
          pred = model(X)
          y = np.reshape(y,(58,1))
          test_loss += loss_fn(pred, y).item()
              
    test_loss /= num_batches
    r2 = r2_score(y, pred) 
    print(f"Test Error: \n r2_scoere: {(100*r2):>0.1f}%, Avg loss: {test_loss:>8f} \n")
     


## Ορισμός υπερπαραμέτρων και εκπαίδευση δικτύου

Κώδικας εκτέλεσης και αξιολόγησης του δικτύου 

In [ ]:
# Το manual_seed καθορίζει την τυχαία αρχικοποίηση των βαρών/παραμέτρων του δικτύου
# Πρόκειται απλώς για ένα σταθερό seed στο random number generator της pytorch.
torch.manual_seed(50)
model = NeuralNetwork1()
learning_rate = 0.001
# Ορίζουμε τη Loss Function
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# Κώδικας εκπαίδευσης εδώ:

for t in range(3000):
    print(f"Epoch {t+1}\n-------------------------------")
    display = t
    train_loop(train_dataloader, model, loss_fn, optimizer, display)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")


Streaming output truncated to the last 5000 lines.
Epoch 2003
-------------------------------
Test Error: 
 r2_scoere: 60.9%, Avg loss: 0.008209 

Epoch 2004
-------------------------------
Test Error: 
 r2_scoere: 60.9%, Avg loss: 0.008206 

Epoch 2005
-------------------------------
Test Error: 
 r2_scoere: 60.9%, Avg loss: 0.008204 

Epoch 2006
-------------------------------
Test Error: 
 r2_scoere: 61.0%, Avg loss: 0.008201 

Epoch 2007
-------------------------------
Test Error: 
 r2_scoere: 61.0%, Avg loss: 0.008198 

Epoch 2008
-------------------------------
Test Error: 
 r2_scoere: 61.0%, Avg loss: 0.008195 

Epoch 2009
-------------------------------
Test Error: 
 r2_scoere: 61.0%, Avg loss: 0.008193 

Epoch 2010
-------------------------------
Test Error: 
 r2_scoere: 61.0%, Avg loss: 0.008190 

Epoch 2011
-------------------------------
Test Error: 
 r2_scoere: 61.0%, Avg loss: 0.008187 

Epoch 2012
-------------------------------
Test Error: 
 r2_scoere: 61.0%, Avg loss: 

Καθώς περναν οι εποχές μπορουμε να παρατηρήσουμε οτι  το r2_score αυξανεται μεχρι να σταθεροποιηθει ενω το avg loss μειωνεται συνεχως οπως και το loss που κανουμε print ανα 100 epochs

Αν αντικαταστήσουμε την γραμμή torch.manual_seed(50) με torch.manual_seed(100) βλεπουμε πως το r2 δεν καταφερε να αυξηθει οσο με το 50 επισης το loss δεν καταφερε να μειωθει τοσο οσο οταν ειχαμε seed 50 

Αν αντικαταστήσουμε τον optimizer από SGD σε Adam βλεπουμε οτι εχουμε πολυ καλυτερα αποτελεσματα και το r2 score καταφερνει να αυξηθει πολυ περισοτερο απο οτι με τον SGD αλλα και το avg loss ειναι πολυ μικροτερο

In [ ]:
from torch.nn.modules.activation import ReLU
# Κώδικας ορισμού του δικτύου
class NeuralNetwork2(nn.Module):
  def __init__(self):
      # Τρέχω την init της κλάσης-γονιού
      super(NeuralNetwork2, self).__init__()
      
      # Δημιουργώ μια μέθοδο Flatten που θα μετατρέπει το Tensor εισόδου σε
      # μονοδιάστατο
      self.flatten = nn.Flatten()

      # Η κλάση Sequential() είναι ένα container μέσα στο οποίο βάζουμε τα
      # Module() που θα χτίζουν τα layers του δικτύου 
      self.network_architecture = nn.Sequential(
          nn.Linear(8,100),
          nn.ReLU(),
          nn.Linear(100,100),
          nn.ReLU(),
          nn.Linear(100,1),
      ) 

  # Η μέθοδος forward απλώς μετατρέπει τα inputs σε διάνυσμα και τα περνάει
  # από τη network_architecture που δημιουργήσαμε
  def forward(self, x):
      x = self.flatten(x)
      logits = self.network_architecture(x)
      return logits


Η εξοδος του δικτυου μας ειναι γραμμικη συνάρτηση των εισόδων

In [ ]:
# Το manual_seed καθορίζει την τυχαία αρχικοποίηση των βαρών/παραμέτρων του δικτύου
# Πρόκειται απλώς για ένα σταθερό seed στο random number generator της pytorch.
torch.manual_seed(50)
model = NeuralNetwork2()
learning_rate = 0.001
# Ορίζουμε τη Loss Function
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# Κώδικας εκπαίδευσης εδώ:

for t in range(3000):
    print(f"Epoch {t+1}\n-------------------------------")
    display = t
    train_loop(train_dataloader, model, loss_fn, optimizer, display)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")


Streaming output truncated to the last 5000 lines.
Epoch 2003
-------------------------------
Test Error: 
 r2_scoere: 69.9%, Avg loss: 0.006322 

Epoch 2004
-------------------------------
Test Error: 
 r2_scoere: 70.1%, Avg loss: 0.006274 

Epoch 2005
-------------------------------
Test Error: 
 r2_scoere: 69.6%, Avg loss: 0.006387 

Epoch 2006
-------------------------------
Test Error: 
 r2_scoere: 70.4%, Avg loss: 0.006219 

Epoch 2007
-------------------------------
Test Error: 
 r2_scoere: 69.4%, Avg loss: 0.006423 

Epoch 2008
-------------------------------
Test Error: 
 r2_scoere: 70.5%, Avg loss: 0.006199 

Epoch 2009
-------------------------------
Test Error: 
 r2_scoere: 69.4%, Avg loss: 0.006431 

Epoch 2010
-------------------------------
Test Error: 
 r2_scoere: 70.3%, Avg loss: 0.006242 

Epoch 2011
-------------------------------
Test Error: 
 r2_scoere: 69.5%, Avg loss: 0.006406 

Epoch 2012
-------------------------------
Test Error: 
 r2_scoere: 70.1%, Avg loss: 

Γενικα αν αλλαξουμε τον sgd σε Adam δεν παρατησουμε μεγαλη διαφορα στην  ταχυτητα αφου ο adam ηταν μολις 2-4 δευτερολεπτα πιο αργος. Ωστωσο στις τιμες βλεπουμε οι με τον adam το r2 αυξανεται παραπανω απο οτι στον sgd αλλα και το avg loss ειναι πιο μικρο